In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('Population_Economic_Indicators.csv')
df


,District_ID,District_Name,Population,Median_Income,Employment_Rate
0,D01,Downtown,45000,72000,0.78
1,D02,Riverside,18500,65000,73%
2,D03,Northgate,22000,NaN,0.69
3,D04,Southridge,15000,59000.5,NaN
4,D05,Westfield,12500,"81,000",0.82
5,D06,Oldtown,8200,47000,65.00%
6,D01,downtown,45200,72500,0.78


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   District_ID      7 non-null      object
 1   District_Name    7 non-null      object
 2   Population       7 non-null      int64 
 3   Median_Income    6 non-null      object
 4   Employment_Rate  6 non-null      object
dtypes: int64(1), object(4)
memory usage: 412.0+ bytes


# Data issues
* Duplicate rows with slightly different values (District_ID D01)
* Inconsistent text name in District_Name column
* Inconsistent data type/format in Median_Income, and Employment_Rate columns.
* Missing values/Null values in Median_Income and Employement_Rate columns.

In [4]:
#Standardize text/format
# Standardize text for District_Name column
df['District_Name'] = (
    df['District_Name'].astype(str).str.strip().str.title()
    )

# Clean Median_Income column, remove ',', change Null to Nan
df['Median_Income']=(
    df['Median_Income'].astype(str).str.replace(',','', regex=False).replace(['NULL', ''], np.nan)
    )
# Convert Median_Income column to numeric
df['Median_Income'] = pd.to_numeric (
    df['Median_Income'], errors='coerce'
)

# Replace nan with the average income
avg_income = df['Median_Income'].mean()
df['Median_Income']=df['Median_Income'].fillna(avg_income).round(2)

df

,District_ID,District_Name,Population,Median_Income,Employment_Rate
0,D01,Downtown,45000,72000.00,0.78
1,D02,Riverside,18500,65000.00,73%
2,D03,Northgate,22000,66083.42,0.69
3,D04,Southridge,15000,59000.50,NaN
4,D05,Westfield,12500,81000.00,0.82
5,D06,Oldtown,8200,47000.00,65.00%
6,D01,Downtown,45200,72500.00,0.78


In [5]:
# Standardize format and handle missing values in Employment rate column
df['Employment_Rate'] = (
    df['Employment_Rate'].astype(str).str.strip().replace(['', 'NULL'], np.nan)
)

def parse_employment_rate(val):
    if pd.isna(val):
        return np.nan
    if '%' in val:
        return float(val.replace('%', ''))/100
    return float(val)

df['Employment_Rate'] = df['Employment_Rate'].apply(parse_employment_rate)
df


,District_ID,District_Name,Population,Median_Income,Employment_Rate
0,D01,Downtown,45000,72000.00,0.78
1,D02,Riverside,18500,65000.00,0.73
2,D03,Northgate,22000,66083.42,0.69
3,D04,Southridge,15000,59000.50,NaN
4,D05,Westfield,12500,81000.00,0.82
5,D06,Oldtown,8200,47000.00,0.65
6,D01,Downtown,45200,72500.00,0.78


In [6]:
# Handle missing value for Employment Rate column 
# Handle possible missing values in numeric columns for future input data (if any).
numeric_col = ['Population', 'Median_Income', 'Employment_Rate']
for col in numeric_col:
    df[col] = df[col].fillna(df[col].mean()).round(2)

df

,District_ID,District_Name,Population,Median_Income,Employment_Rate
0,D01,Downtown,45000,72000.00,0.78
1,D02,Riverside,18500,65000.00,0.73
2,D03,Northgate,22000,66083.42,0.69
3,D04,Southridge,15000,59000.50,0.74
4,D05,Westfield,12500,81000.00,0.82
5,D06,Oldtown,8200,47000.00,0.65
6,D01,Downtown,45200,72500.00,0.78


In [7]:
# Manage duplicate rows. 
'''Aggregate rows with the same District_ID,
but with slightly different values, to keep one row with average values.'''
df = (df.groupby(['District_ID', 'District_Name'], as_index=False)
        .agg ({'Population': 'mean',
                'Median_Income': 'mean',
                'Employment_Rate': 'mean'})
                )
df

,District_ID,District_Name,Population,Median_Income,Employment_Rate
0,D01,Downtown,45100.0,72250.00,0.78
1,D02,Riverside,18500.0,65000.00,0.73
2,D03,Northgate,22000.0,66083.42,0.69
3,D04,Southridge,15000.0,59000.50,0.74
4,D05,Westfield,12500.0,81000.00,0.82
5,D06,Oldtown,8200.0,47000.00,0.65


In [8]:
# Population column values should remain as integer 
df['Population'] = pd.to_numeric(
    df['Population'], downcast='integer', errors='coerce'
    )

In [9]:
# Testing and export the cleaned data
df.info()
df.to_csv('Population_Economic_Indicators_output.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   District_ID      6 non-null      object 
 1   District_Name    6 non-null      object 
 2   Population       6 non-null      int32  
 3   Median_Income    6 non-null      float64
 4   Employment_Rate  6 non-null      float64
dtypes: float64(2), int32(1), object(2)
memory usage: 348.0+ bytes
